In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import *
from bokeh.models import *
import bokeh.palettes as bp
from bokeh.resources import INLINE
from bokeh.io import output_notebook, show
# output_notebook(resources=INLINE)
output_notebook()
%matplotlib inline

Loading BokehJS ...

In [2]:
# Load data from csv file
crimes = pd.read_csv("crime.csv")
atlanta = pd.read_csv("atlanta_crime.csv",parse_dates=True)
crashes = pd.read_csv("all-crashes2004-2016.csv")
crashes = crashes.fillna(0)
# accidents = pd.read_csv("accidents_2012_to_2014.csv")
# w = pd.read_csv("Washington_State_HDMA-2016.csv")
# count crimes by location neighborhood
crimes_area_count = crimes.groupby('INCIDENTNEIGHBORHOOD')['INCIDENTNEIGHBORHOOD'].count().reset_index(name="count")
# sort by crimes count
crimes_area_count = crimes_area_count.sort_values(by=['count'], ascending = False)
print(crimes_area_count.head())
print(list(crashes))
crashes.head()

/Users/liuchang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (79,183) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


         INCIDENTNEIGHBORHOOD  count
16  Central Business District    260
74           South Side Flats    193
15                    Carrick    125
28               East Liberty    115
70                  Shadyside     80
['CRASH_CRN', 'DISTRICT', 'CRASH_COUNTY', 'MUNICIPALITY', 'POLICE_AGCY', 'CRASH_YEAR', 'CRASH_MONTH', 'DAY_OF_WEEK', 'TIME_OF_DAY', 'HOUR_OF_DAY', 'ILLUMINATION', 'WEATHER', 'ROAD_CONDITION', 'COLLISION_TYPE', 'RELATION_TO_ROAD', 'INTERSECT_TYPE', 'TCD_TYPE', 'URBAN_RURAL', 'LOCATION_TYPE', 'SCH_BUS_IND', 'SCH_ZONE_IND', 'TOTAL_UNITS', 'PERSON_COUNT', 'VEHICLE_COUNT', 'AUTOMOBILE_COUNT', 'MOTORCYCLE_COUNT', 'BUS_COUNT', 'SMALL_TRUCK_COUNT', 'HEAVY_TRUCK_COUNT', 'SUV_COUNT', 'VAN_COUNT', 'BICYCLE_COUNT', 'FATAL_COUNT', 'INJURY_COUNT', 'MAJ_INJ_COUNT', 'MOD_INJ_COUNT', 'MIN_INJ_COUNT', 'UNK_INJ_DEG_COUNT', 'UNK_INJ_PER_COUNT', 'UNBELTED_OCC_COUNT', 'UNB_DEATH_COUNT', 'UNB_MAJ_INJ_COUNT', 'BELTED_DEATH_COUNT', 'BELTED_MAJ_INJ_COUNT', 'MCYCLE_DEATH_COUNT', 'MCYCLE_MAJ_INJ

,CRASH_CRN,DISTRICT,CRASH_COUNTY,MUNICIPALITY,POLICE_AGCY,CRASH_YEAR,CRASH_MONTH,DAY_OF_WEEK,TIME_OF_DAY,HOUR_OF_DAY,...,ACCESS_CTRL,ROADWAY_COUNTY,LANE_COUNT,RDWY_ORIENT,ROAD_OWNER,ROUTE,SPEED_LIMIT,SEGMENT,OFFSET,STREET_NAME
0,2.004000e+09,11.0,2.0,2454.0,02454,2004.0,1.0,5.0,257.0,2.0,...,3.0,2.0,2.0,E,2.0,2032,35.0,10.0,1000.0,LEWIS RUN RD
1,2.004000e+09,11.0,2.0,2440.0,02440,2004.0,1.0,5.0,1027.0,10.0,...,0.0,2.0,2.0,W,3.0,0,35.0,0.0,0.0,COAL VALLEY RD
2,2.004000e+09,11.0,2.0,2203.0,02203,2004.0,1.0,5.0,1839.0,18.0,...,0.0,2.0,2.0,W,2.0,3082,35.0,0.0,0.0,CLINTON FRANKFORT RD
3,2.004000e+09,11.0,2.0,2404.0,02404,2004.0,1.0,6.0,914.0,9.0,...,0.0,2.0,2.0,N,4.0,0,25.0,0.0,0.0,NORTH BALPH AV
4,2.004000e+09,11.0,2.0,2419.0,02419,2004.0,1.0,5.0,2230.0,22.0,...,0.0,2.0,2.0,S,4.0,0,20.0,0.0,0.0,THORNBERRY DR


In [3]:
crimes['Date'] = pd.to_datetime(crimes['INCIDENTTIME'])
crimes['Year'] = crimes['Date'].dt.year
crimes['Month'] = crimes['Date'].dt.month
crimes['Hour'] = crimes['Date'].dt.hour
crimes.head()

,_id,PK,CCR,HIERARCHY,INCIDENTTIME,INCIDENTLOCATION,CLEAREDFLAG,INCIDENTNEIGHBORHOOD,INCIDENTZONE,INCIDENTHIERARCHYDESC,OFFENSES,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y,Date,Year,Month,Hour
0,1,173911934,18043229,10,2018-03-07T11:11:00,"900 Block Berkshire Ave Pittsburgh, PA 15226",Y,Brookline,6,HARRASSMENT,2709 Harassment (Non-Specific),1918.0,NaN,NaN,0.000000,0.000000,2018-03-07 11:11:00,2018,3,11
1,2,-3025023287,18030094,99,2018-02-16T09:10:00,"4200 Block 5th Ave Pittsburgh, PA 15213",N,North Oakland,4,INDUSTRIAL ACCIDENT,9491 Industrial Accident,403.0,NaN,NaN,-79.953193,40.444302,2018-02-16 09:10:00,2018,2,9
2,3,173112511,18034373,0,2018-02-22T11:48:00,"Perrysville Ave & Delaware St Pittsburgh, PA 1...",N,Perry North,1,99,8106 Shots Fired,2602.0,NaN,NaN,-80.011790,40.479682,2018-02-22 11:48:00,2018,2,11
3,4,173116418,18034380,0,2018-02-22T12:05:00,"3100 Block Arlington Ave Pittsburgh, PA 15210",N,Arlington Heights,3,99,9999 9999 - (Misc. Incident):,1604.0,NaN,NaN,-79.964106,40.417531,2018-02-22 12:05:00,2018,2,12
4,5,172566354,18028506,10,2018-02-14T01:52:00,"700 Block Johnston Ave Pittsburgh, PA 15207",Y,Glen Hazel,4,SIMPLE ASSAULT,2701 Simple Assault. / 2606(a) Aggravated Assa...,1504.0,NaN,NaN,-79.933411,40.405788,2018-02-14 01:52:00,2018,2,1


In [4]:
from numpy import linspace
x = linspace(0, 20, 20)
# create a new plot (with a title) using figure
p = figure(plot_width=400, plot_height=400, title="Line Plot Example")
# add a line renderer
p.line(x, y = crimes["HIERARCHY"][:20], line_width=2, legend="incident hierrarchy")
p.add_tools(HoverTool(tooltips=[("number", "@y")]))
show(p) # show the results

In [5]:
INCIDENTNEIGHBORHOOD = list(crimes_area_count['INCIDENTNEIGHBORHOOD'][:30])
p = figure(y_range=INCIDENTNEIGHBORHOOD,plot_width=800, 
           plot_height=500, title="Incident count by area")
p.hbar(y=INCIDENTNEIGHBORHOOD, right=list(crimes_area_count['count'][:30]), 
       height=0.8,left=0, color=bp.Viridis256[40:70])
p.add_tools(HoverTool(tooltips=[("incident number", "@right")]))
show(p)